## The main motive of this notebook is to understand the big difference in create models with and without using pipelines.

In [1]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier


In [2]:
path = 'C:/Users/himan/OneDrive/Desktop/Titanic-Dataset.csv'

df = pd.read_csv(path)

In [3]:
df.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
724,725,1,1,"Chambers, Mr. Norman Campbell",male,27.0,1,0,113806,53.1000,E8,S
353,354,0,3,"Arnold-Franchi, Mr. Josef",male,25.0,1,0,349237,17.8000,NaN,S
629,630,0,3,"O'Connell, Mr. Patrick D",male,NaN,0,0,334912,7.7333,NaN,Q
236,237,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26707,26.0000,NaN,S
128,129,1,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,F E69,C
719,720,0,3,"Johnson, Mr. Malkolm Joackim",male,33.0,0,0,347062,7.7750,NaN,S
527,528,0,1,"Farthing, Mr. John",male,NaN,0,0,PC 17483,221.7792,C95,S
80,81,0,3,"Waelens, Mr. Achille",male,22.0,0,0,345767,9.0000,NaN,S
516,517,1,2,"Lemore, Mrs. (Amelia Milley)",female,34.0,0,0,C.A. 34260,10.5000,F33,S
463,464,0,2,"Milling, Mr. Jacob Christian",male,48.0,0,0,234360,13.0000,NaN,S


In [4]:
#Droping columns which are of no use.
df.drop(columns=['Name','PassengerId','Ticket','Cabin'],inplace=True)

In [5]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [6]:
#step-1 ------> train test split
# spliting depandent and indepandent varibles. 
x = df.iloc[:,1:]
y = df.iloc[:,0]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)



In [7]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [8]:
y_train.shape


(712,)

In [9]:
x_test.shape

(179, 7)

In [10]:
y_test.shape

(179,)

In [11]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [12]:
# APPLYING IMPUTATION

si_age = SimpleImputer()
si_emb = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_emb = si_emb.fit_transform(x_train[['Embarked']])


x_test_age = si_age.fit_transform(x_test[['Age']])
x_test_emb = si_emb.fit_transform(x_test[['Embarked']])



In [13]:
## APPLYING OneHotEncoding On Sex and Embarked
# no need remove Multicollinearity to drop first column because we will use DecisionTree 
ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_emb = OneHotEncoder(sparse=False,handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_emb = ohe_emb.fit_transform(x_train_emb)


x_test_sex = ohe_sex.fit_transform(x_test[['Sex']])
x_test_emb = ohe_emb.fit_transform(x_test_emb)


In [14]:
x_train_sex

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [15]:
x_train_emb

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [16]:
x_train_rem =x_train.drop(columns  = ['Sex','Age','Embarked'])
x_test_rem = x_test.drop(columns  = ['Sex','Age','Embarked'])

In [17]:
x_train_transformed = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_emb),axis = 1)
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_emb),axis = 1)

In [18]:
x_train_transformed.shape

(712, 10)

In [19]:
x_test_transformed.shape

(179, 10)

In [20]:
clf  = DecisionTreeClassifier()
clf.fit(x_train_transformed,y_train)


DecisionTreeClassifier()

In [21]:
y_pred = clf.predict(x_test_transformed)

In [22]:
y_pred # survied or not.

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1], dtype=int64)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7374301675977654

In [24]:
import pickle

In [26]:
pickle.dump(ohe_sex,open('Model/ohe_sex.pkl','wb'))
pickle.dump(ohe_emb,open('Model/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('Model/clf.pkl','wb'))

In [27]:
ohe_sex = pickle.load(open('Model/ohe_sex.pkl','rb'))
ohe_emb = pickle.load(open('Model/ohe_embarked.pkl','rb'))
clf = pickle.load(open('Model/clf.pkl','rb'))

In [28]:
# Assume user input
# Pclass/gender/age/SibSp/Parch/Fare/Embarked
test_input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [29]:
test_input


array([[2, 'male', 31.0, 0, 0, 10.5, 'S']], dtype=object)

In [31]:
test_input_sex = ohe_sex.transform(test_input[:,1].reshape(1,1))


C:\Users\himan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [32]:
test_input_sex

array([[0., 1.]])

In [33]:
test_input_emb = ohe_emb.transform(test_input[:,-1].reshape(1,1))


In [34]:
test_input_emb

array([[0., 0., 1.]])

In [35]:
test_input_age = test_input[:,2].reshape(1,1)

In [36]:
test_input_transformed = np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_emb),axis=1)


In [37]:
test_input_transformed.shape

(1, 10)

In [38]:
clf.predict(test_input_transformed)

array([1], dtype=int64)

### THIS IS SO COMPLEXED AS PAINFULL?? ITS HARD PUTTING MODELS IN USE WITHOUT USING PIPELINES RIGHT??